In [17]:
import numpy as np
import torch
import math

In [18]:
class makeCoords(torch.nn.Module):
    def __init__(self):
        """
        Layer to make the coordiante pairs
        The self.coords parameter is what we care about training in the Net so we initialize it here
        and then we'll continue passing the self.coords Param (as X) through the Net
        """
        super().__init__()
        self.coords = torch.nn.Parameter(torch.tensor([1,2,3,4,5,6,7,8], dtype = torch.float32))
        
    def forward(self, x):
        print("    New Coordinates: ", [i.item() for i in self.coords])
        # We have to do ~something~ to the Param here, I think of it kind of like 'activating' the parameter
        # We can also add or subtract to 'activate' but this just returns the self.coords without any changes
        return 1 * self.coords

In [19]:
class SocialSig(torch.nn.Module):
    '''
    Class to create the social signature image
    '''
    def __init__(self):
        '''
        Basic steps for class:
            1. Randomly initalize 'weights' which I believe are actually the coords of the points
            2. Train kriggin model to interpolate points in between
            3. Predict what the points would be within a 224x224 matrix and output the resulting matrix
        '''
        super(SocialSig, self).__init__()
        self.outDim = [10,10]
        self.padding = torch.nn.ConstantPad1d((0,92), 1)

    def forward(self, inputs, x):
        
        self.X = inputs
        
        # Create our blank grid that we will fill with the IDW values
        self.grid = self.__make_blank_coord_grid()
        
        # I did this moslty because it was easier to create a var called 'coords' then to replace 
        # every instance of it with x. Using clone preserves the trainable parameters, so that we 
        # can eventually return coords instead of x from this layer
        coords = x.clone()
        
        # Get the 'image' from IDW
        tensorRet = self.IDW(coords, inputs)
        
        # Flatten the IDW image into tensor list
        tensorRet = tensorRet.flatten()
        
        # Pad the image with 1's (this is to get it into the final shape of the iamge.)
        # At this point we are done with the actual coord values but we still need the 'shell' of the 
        # parameter so we are reshaping it into what we need for the final output
        coords = self.padding(coords)
        
        
        # I need to fix something here but we have two matrices: 
        #    1) 'coords' is the original coords matrix that we just padded with a bunch of 1's 
        #    2) 'tensorRet' which has all of the IDW values
        # They're the same shape so we just multiply in order to 'replace' the value in our coords tensor
        # with those from our tensorRet. The output from this is our image that we then keep feeding through 
        # the remaining layers in the forward function within the Net
        # TO FIX: The first 8 values of coords are still the parameterized coords so you need to zero them out first
        return coords * tensorRet
       
    
    def IDW(self, coords, inputs):
        '''
        Train the IDW model to predict all of the points that are between known points
        '''
        coords = torch.clamp(coords, min=0, max=self.outDim[1])
                
        for cell in range(0, len(self.grid)):
            weightedVals = []
            for column in range(0, len(inputs)):
                xCoordLookup = column * 2
                yCoordLookup = xCoordLookup + 1
            
                measurementCellValue = self.X[column]

                estCellX = self.grid[cell][0]
                estCellY = self.grid[cell][1]

                measureCellX = coords[xCoordLookup]
                measureCellY = coords[yCoordLookup]

                A2 = abs(estCellX - measureCellX)**2
                B2 = abs(estCellY - measureCellY)**2
                C2 = math.sqrt(A2+B2) 
                if(C2 == 0):
                    C2 = 1
                
                measurementCellValue = [measurementCellValue.numpy().tolist()]
                weightedVals.append(measurementCellValue[0] * (1/(C2**2)))
            self.grid[cell] = sum(weightedVals)
        numpyGrid = torch.from_numpy(np.reshape(np.array(self.grid), (1,1,self.outDim[0],self.outDim[1])))
        tensorGrid = torch.tensor(numpyGrid, dtype=torch.float)

        return tensorGrid 
        
    def __make_blank_coord_grid(self):
        '''
        Make a blank coordinate grid to fill in with real data later
        '''
        outDim = [10, 10]
        return [[x,y] for x in range(0, outDim[0]) for y in range(0,outDim[1])]

In [20]:
class SocialSigNet(torch.nn.Module):
    def __init__(self):
        """
        In the constructor we instantiate four parameters and assign them as
        member parameters.
        """
        super().__init__()
        self.makeCoords = makeCoords()               # Layer to make the coordinate pairs
        self.linear2 = torch.nn.Linear(10, 1)        # Linear Layer
        self.SocialSig = SocialSig()                 # SocialSig layer that creates the nxn image
        self.conv1 = torch.nn.Conv2d(1, 10, 10, 1)   # 1-D convolutional layer


    def forward(self, x, inputs):
        """
        In the forward function we accept a Tensor of input data and we must return
        a Tensor of output data. We can use Modules defined in the constructor as
        well as arbitrary operators on Tensors.
        - We have to keep x as the original training param the whole time, so only
        perform transformations on the variable x, which begins as a set of randomly 
        initalized points, then gets transformed into an image.
        """

        # Here we make our coordinate list
        x = self.makeCoords(x)
        x = self.SocialSig(inputs, x)
        
        # Here we reshape it into an imagy shape (not sure if this is totally neccessary)
        x = torch.reshape(x, (1, 1, 10, 10))
        x = self.conv1(x)
        
        x = torch.flatten(x)
        x = self.linear2(x)
        
        return x

In [21]:
# Create random tensors to hold input and outputs.
x = torch.tensor([-1.6848])
y = torch.tensor([0.0359, 0.7196, 0.7006, 7.2164, 5.4166, 0.4062, 2.2501, 6.6463])

# Construct our model by instantiating the class defined above
model = SocialSigNet()

In [22]:

criterion = torch.nn.MSELoss(reduction='sum')
optimizer = torch.optim.SGD(model.parameters(), lr=1e-5)
for t in range(5):
    
    print("Epoch: ", t)
    
    # Forward pass: Compute predicted y by passing x to the model
    # I pass the inputs twice -- This might be fixable later (after sleep) but the idea was that the first
    # x param gets immediately reset to the coordinate 'weights' and the second is preserved until we need it 
    # for the IDW
    # ^^ I think I know a better way to do that but for now...
    y_pred = model(x, x)
    
    print("    Predicted Y: ", y_pred)
    
    # Compute and print loss
    # This autograd stuff is seeming to me vaguely like recursion. It seems like the parameters that will
    # be updated in loss.backward() are those that are directly returned by the forward pass (i.e. the 
    # params linked to y_pred)
    loss = criterion(y_pred, y)
    print("    Loss: ", loss)
    if t % 100 == 99:
        print(t, loss.item())

    # Zero gradients, perform a backward pass, and update the weights.
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    print("\n")


Epoch:  0
    New Coordinates:  [1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0]
    Predicted Y:  tensor([-0.4477], grad_fn=<AddBackward0>)
    Loss:  tensor(154.3757, grad_fn=<MseLossBackward>)


Epoch:  1
    New Coordinates:  [0.9999999403953552, 1.999999761581421, 2.9999959468841553, 3.9999990463256836, 4.999999523162842, 6.0, 7.0, 8.0]
    Predicted Y:  tensor([1.5086e+12], grad_fn=<AddBackward0>)
    Loss:  tensor(1.8208e+25, grad_fn=<MseLossBackward>)


Epoch:  2
    New Coordinates:  [16829.71484375, 123585.65625, 1782109.0, 455306.46875, 140058.34375, 22846.23046875, 4027.679931640625, 29038.208984375]
    Predicted Y:  tensor([2.8492e+38], grad_fn=<AddBackward0>)
    Loss:  tensor(inf, grad_fn=<MseLossBackward>)


Epoch:  3
    New Coordinates:  [nan, nan, nan, nan, nan, nan, nan, nan]
    Predicted Y:  tensor([nan], grad_fn=<AddBackward0>)
    Loss:  tensor(nan, grad_fn=<MseLossBackward>)


Epoch:  4
    New Coordinates:  [nan, nan, nan, nan, nan, nan, nan, nan]
    Predicted Y:  t

/Users/heatherbaier/anaconda/envs/caoe/lib/python3.6/site-packages/ipykernel_launcher.py:80: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
